In [3]:
import numpy as np 
from Algorithms.sudoku import Individual

In [35]:
board = np.array([[3, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 9, 0, 9, 4, 0, 0],
       [0, 0, 8, 0, 0, 7, 3, 8, 0],
       [0, 7, 0, 5, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 2, 3],
       [2, 0, 0, 0, 0, 0, 0, 0, 4],
       [0, 5, 6, 0, 0, 0, 0, 0, 6],
       [0, 0, 2, 0, 0, 0, 3, 0, 1]])
new_individual = Individual(initial_board=board, board=board)

In [36]:
new_individual.get_possible_numbers((6,0))

array([7, 8, 9])